### **basic workfloww: compare runs, choose a model, deploy it to rest api**

- in this quickstart we will be:

1. run a hyperparameter swap on trainign script
2. will compare results of run in mlflow ui
3. choose best run and register it as a model
4. deploy model to rest ap
5.  build a container image suitable for deployment to cloud platform
6. will be perofrming mulitple experiments

In [11]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe#used for performing hyperParameter tuning
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [7]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [8]:
## splitting the dataset

train, test=train_test_split(data, test_size=0.25, random_state=42)

In [9]:
train_x=train.drop(['quality'], axis=1).values
train_y=train[['quality']].values.ravel()

#test set
test_x=test.drop(['quality'], axis=1).values
test_y=test[['quality']].values.ravel()

#splitting this train into train and vlidation
train_x, valid_x, train_y, valid_y=train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature=infer_signature(train_x, train_y)

In [12]:
np.mean(train_x, axis=0)

array([6.86621852e+00, 2.80377808e-01, 3.32597005e-01, 6.42164738e+00,
       4.55513955e-02, 3.53556841e+01, 1.38792376e+02, 9.94074221e-01,
       3.18919333e+00, 4.88396869e-01, 1.05005673e+01])

## **creating our very own ANN Model**

In [21]:
## model creation
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    #defining our model architecture
    mean=np.mean(train_x, axis=0) #we are calculating mean column wise
    var=np.var(train_x, axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1) # 1 output model
        ]
    )

    ## compiling our model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ), 
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## training our model ANN with lr and momentum params with MLFlwo tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                    epochs=epochs, 
                    batch_size=64
                )
        
        ## evaluating the model
        eval_result=model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse=eval_result[1]

        ## logging the parameters
        mlflow.log_params(params)
        mlflow.log_metric("eval_rsme", eval_rmse)

        ## logging the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}
        



In [22]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [23]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)
    }

In [24]:
mlflow.set_experiment('/wine_quality')
with mlflow.start_run():
    #conducting the hyperparameter search using hyperopt
    trials=Trials()
    best=fmin(
        fn=objective, 
        space=space, 
        algo=tpe.suggest, 
        max_evals=4, 
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/01/08 13:22:53 INFO mlflow.tracking.fluent: Experiment with name '/wine_quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 23s 518ms/step - loss: 40.1655 - root_mean_squared_error: 6.3376
38/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 38.9879 - root_mean_squared_error: 6.2440   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 38.9408 - root_mean_squared_error: 6.2402 - val_loss: 38.2478 - val_root_mean_squared_error: 6.1845

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 39.7152 - root_mean_squared_error: 6.3020
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.1768 - root_mean_squared_error: 6.1787 - val_loss: 37.7358 - val_root_mean_squared_error: 6.1429

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 35.6148 - root_mean_squared_error: 5.9678
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.6008 - root_mean_squared_error: 6.1319 - val_loss: 37.2318 - val_root_mean_squared_error: 6.1018

 1/12 ━━━━━━━━━━━━━━━━━━━━ 

## **inferencing the model**

In [26]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/e5e5ccb265e14a72981c5bd4c216fda8/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\Piyush\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.671341 ],
       [7.189119 ],
       [6.192364 ],
       ...,
       [6.5339627],
       [6.77333  ],
       [5.284866 ]], dtype=float32)

In [27]:
import mlflow
logged_model = 'runs:/e5e5ccb265e14a72981c5bd4c216fda8/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[5.671341 ],
       [7.189119 ],
       [6.192364 ],
       ...,
       [6.5339627],
       [6.77333  ],
       [5.284866 ]], dtype=float32)

In [28]:
## registering them ode
mlflow.register_model(model_uri, "wine-quality-final-model")

Successfully registered model 'wine-quality-final-model'.
Created version '1' of model 'wine-quality-final-model'.


<ModelVersion: aliases=[], creation_timestamp=1736324268384, current_stage='None', description=None, last_updated_timestamp=1736324268384, name='wine-quality-final-model', run_id='e5e5ccb265e14a72981c5bd4c216fda8', run_link=None, source='file:///c:/Users/Piyush/Desktop/Personal%20Work/MLOps-Tutorials/Deep%20Learning%20ANN%20Model/mlruns/893108091327265436/e5e5ccb265e14a72981c5bd4c216fda8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>